In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

from tensorflow import keras

In [2]:
model = keras.models.load_model("models/TA98_7_28_23.h5")

In [3]:
def parse(input_string, target_rows=15):
    positions = input_string.split("%")

    chunk_df = pd.DataFrame([list(map(float, pos.split(","))) for pos in positions], columns=["x", "y", "z"])

    if len(chunk_df) == target_rows:
        return chunk_df

    linspace = pd.DataFrame(
        index=pd.RangeIndex(target_rows),
        columns=chunk_df.columns
    )

    for col in ["x", "y", "z"]:
        linspace[col] = pd.Series(np.interp(
            np.linspace(0, 1, num=target_rows),
            np.linspace(0, 1, num=len(chunk_df)),
            chunk_df[col]
        ))

    return linspace.values.tolist()

In [12]:
data_string = "4.5,3.51,-3.75%4.5,3,-3.75%4.5,3,-3.75%4.5,3,-3.75"

custom_test_positions = parse(data_string)
custom_test_positions = np.array(custom_test_positions)

prediction = model.predict(custom_test_positions.reshape(-1, 15, 3))[0][0]
print(f"raw prediction: {prediction}")
print(f"rounded: {np.round(prediction)}")

is_exploiting = np.round(prediction) == 1

print(f"was player exploting? {is_exploiting}")

1/1 [==============================] - 0s 25ms/step
raw prediction: 0.9882161021232605
rounded: 1.0
was player exploting? True
